In [1]:
import pandas as pd
import json

def write_json(output_json, path):
    json_object = json.dumps(output_json, indent=4)
    with open(path, "w") as outfile:
        outfile.write(json_object)
        
def extract_data(df):
    return [
        {
            "input": row["source"],
            "labels": [row["target"]],
            "report_id": row["doi"],
        }
        for i, row in df.copy().iterrows()
    ]

In [2]:
with open('../NapSS/data/data-tico19/test.doi') as f:
    doi = f.readlines()
with open('../NapSS/data/data-tico19/test.source') as f:
    source = f.readlines()
with open('../NapSS/data/data-tico19/test.target') as f:
    target = f.readlines()

In [3]:
df = pd.DataFrame({'doi':doi, 'source':source, 'target':target})

In [4]:
df = df.applymap(lambda x: x.strip('\n'))

In [5]:
df = df.loc[df.source.apply(lambda s: s[-1] in ['.','?','!'])]

In [ ]:
set(df['doi'].apply(lambda x: x.split(':')[0]))

In [ ]:
list(df.loc[df.doi.str.contains('PubMed')|df.doi.str.contains('Wikipedia_handpicked_')|df.doi.str.contains('wiki_')].index)

In [29]:
import argparse
from datasets import load_dataset
from utils_eval import compute_metrics

# Get dataset from arguments

DATASET_NAME = "tico19"
dataset = load_dataset(
    "json", data_files=f"data/{DATASET_NAME}_multiple.json", field="train"
)
dataset["test"] = load_dataset(
    "json", data_files=f"data/{DATASET_NAME}_multiple.json", field="test"
)["train"]

sources = [item["input"] for item in dataset["test"]]
labels = [item["labels"] for item in dataset["test"]]


dois = [item["id"] for item in dataset["test"]]


Found cached dataset json (/home/lyf6/.cache/huggingface/datasets/json/default-3df652e8f5701e6a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (/home/lyf6/.cache/huggingface/datasets/json/default-75d6e0dce475b45a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

KeyError: 'id'

In [51]:
df_json = extract_data(df)

In [53]:
output_json = {"train": df_json, "test": df_json}

In [56]:
write_json(output_json, "data/tico19.json")
write_json(output_json, "data/tico19_multiple.json")